In [ ]:
import sys
!{sys.executable} -m pip install pycbc ligo-common --no-cache-dir

     |████████████████████████████████| 6.7 MB 4.7 MB/s 
     |████████████████████████████████| 27.4 MB 1.1 MB/s 
     |████████████████████████████████| 201 kB 77.9 MB/s 
     |████████████████████████████████| 75 kB 44.8 MB/s 
     |████████████████████████████████| 1.6 MB 28.3 MB/s 
     |████████████████████████████████| 51 kB 56.3 MB/s 
     |████████████████████████████████| 55 kB 45.0 MB/s 
     |████████████████████████████████| 3.6 MB 49.5 MB/s 
  Created wheel for lscsoft-glue: filename=lscsoft_glue-2.0.0-cp37-cp37m-linux_x86_64.whl size=416860 sha256=4a29dce6ee230d3a6ce40efbc6b7ddfbbdccde4304309e83cddbdd1a787b217f
  Stored in directory: /tmp/pip-ephem-wheel-cache-e4b52btg/wheels/9d/ed/9e/1f9cad5e15359dc17954b893dd486760b5cb1ffd97a8405d1e
  Created wheel for ligo-segments: filename=ligo_segments-1.4.0-cp37-cp37m-linux_x86_64.whl size=86664 sha256=08d53db5c4255b9733269b5dbd040699f198a4dfc2191c006d2cfe38f764072d
  Stored in directory: /tmp/pip-ephem-wheel-cache-e4b52btg/wheels

In [ ]:
import numpy as np
import math
import pylab

import matplotlib.pyplot as plt
import random
import pycbc
from pycbc import distributions
from pycbc.waveform import get_td_waveform
from pycbc.detector import Detector
import pycbc.coordinates as co
from pycbc.psd import welch, interpolate
from pycbc.psd import interpolate, inverse_spectrum_truncation
from pycbc.noise.gaussian import noise_from_psd
from pycbc.noise.gaussian import frequency_noise_from_psd
from pycbc.filter import matched_filter

det_l1 = Detector('L1')
apx = 'IMRPhenomD'
N=2048*16  #N is number of samples, N=length/delta_t
fs=2048 #fs is sampling frequnecy
length=16 #duration of segment
delta_f=1.0/16
f_samples = 16385
f_lower=30
delta_t=1.0/2048

# N=2048*16.0  #N is number of samples, N=length/delta_t
# print(N)
# fs=2048.0 #fs is sampling frequnecy
# print(fs)
# length=16 #duration of segment
# print(length)
# delta_f=fs/N
# print(delta_f)
# f_samples = int(N/2+ 1)
# print(f_samples)
# f_lower=40
# print(f_lower)
# delta_t=1.0/2048
# print(delta_t)

In [ ]:
from pycbc.psd.analytical import AdVDesignSensitivityP1200087

def get_psd(f_samples, delta_f, low_freq_cutoff):
    psd=AdVDesignSensitivityP1200087(f_samples, delta_f, low_freq_cutoff)
    return psd

from pycbc.noise.gaussian import frequency_noise_from_psd

def get_noise(psd, seed=None):
    noise=frequency_noise_from_psd(psd, seed=seed)
    noise_time = noise.to_timeseries()
    return noise_time

def add_noise_signal(noise, signal):
    length_signal = len(signal)
    signal_plus_noise=noise
    signal_plus_noise[0:length_signal]=np.add(noise[0:length_signal], signal)
    return signal_plus_noise


from pycbc.psd import welch, interpolate

def get_whiten(signal_plus_noise):
    signal_freq_series=signal_plus_noise.to_frequencyseries()
    numerator = signal_freq_series
    psd_to_whiten = interpolate(welch(signal_plus_noise), 1.0 / signal_plus_noise.duration)
    denominator=np.sqrt(psd_to_whiten)
    whiten_freq = (numerator / denominator)
    whiten=whiten_freq.to_timeseries().highpass_fir(30., 512).lowpass_fir(300.0, 512)
    return whiten

def get_8s(whiten, signal_peak_index=None):
    whiten.start_time = 0
    cropped = whiten.time_slice(0,8)
    return cropped

psd=get_psd(f_samples, delta_f, f_lower)

def DISTRIBUTIONS(low, high, samples):
    var_dist = distributions.Uniform(var = (low, high))
    return var_dist.rvs(size = samples)

def SPIN_DISTRIBUTIONS(samples):
    theta_low = 0.
    theta_high = 1.
    phi_low = 0.
    phi_high = 2.
    uniform_solid_angle_distribution = distributions.UniformSolidAngle(polar_bounds=(theta_low,theta_high),
                                              azimuthal_bounds=(phi_low,phi_high))
    solid_angle_samples = uniform_solid_angle_distribution.rvs(size=samples)
    spin_mag = np.ndarray(shape=(samples), dtype=float)
    for i in range(0,samples):
        spin_mag[i] = 1.
    spinx, spiny, spinz = co.spherical_to_cartesian(spin_mag,solid_angle_samples['phi'],solid_angle_samples['theta'])
    return spinz

In [ ]:
def get_params(samples):
    mass1_samples = DISTRIBUTIONS(10, 80, samples)
    mass2_samples = DISTRIBUTIONS(10, 80, samples)
    right_ascension_samples  = DISTRIBUTIONS(0 , 2*math.pi, samples)
    polarization_samples = DISTRIBUTIONS(0 , 2*math.pi, samples)
    declination_samples = DISTRIBUTIONS((-math.pi/2)+0.0001, (math.pi/2)-0.0001, samples)
    spinz1 = SPIN_DISTRIBUTIONS(samples)
    spinz2 = SPIN_DISTRIBUTIONS(samples)
    snr_req = DISTRIBUTIONS(2, 17, samples)
    DIST = DISTRIBUTIONS(2500, 3000, samples)
    return mass1_samples, mass2_samples, right_ascension_samples, polarization_samples, declination_samples, spinz1, spinz2, snr_req, DIST

In [ ]:
def DATA_GENERATION(samples):

  mass1_samples, mass2_samples, right_ascension_samples, polarization_samples, declination_samples, spinz1, spinz2, snr_req, DIST = get_params(samples)
  for i in range(0,samples):
        seed =  random.randint(1, 256)
        # NOTE: Inclination runs from 0 to pi, with poles at 0 and pi
        #       coa_phase runs from 0 to 2 pi.
        try:
          hp, hc = get_td_waveform(approximant=apx,
                                  mass1=mass1_samples[i][0],
                                  mass2=mass2_samples[i][0],
                                  spin1z=spinz1[i],
                                  spin2z=spinz2[i],
                                  delta_t=delta_t,
                                  distance = DIST[i][0],
                                  f_lower=40)
        except:
          try:
            hp, hc = get_td_waveform(approximant=apx,
                          mass1=mass1_samples[i][0],
                          mass2=mass2_samples[i][0],
                          spin1z=spinz1[i],
                          spin2z=spinz2[i],
                          delta_t=delta_t,
                          distance = DIST[i][0],
                          f_lower=50)
          except RuntimeError:
            hp, hc = get_td_waveform(approximant=apx,
                                  mass1=mass1_samples[i][0],
                                  mass2=mass2_samples[i][0],
                                  spin1z=spinz1[i],
                                  spin2z=spinz2[i],
                                  delta_t=delta_t*2,
                                  distance = DIST[i][0],
                                  f_lower=40)
             

        signal_l1 = det_l1.project_wave(hp, hc,  right_ascension_samples[i][0], declination_samples[i][0], polarization_samples[i][0])
        signal_l1.append_zeros(10*2048)
        signal_l1 = signal_l1.cyclic_time_shift(5)
        signal_l1.start_time = 0 

        noise=get_noise(psd)
        final = add_noise_signal(noise, signal_l1)

        hps=signal_l1
        conditioned=final
        hps.resize(len(conditioned))
        template = hps.cyclic_time_shift(hps.start_time)
        psd_whiten=interpolate(welch(conditioned), 1.0 / conditioned.duration)
        snr = matched_filter(template, conditioned, psd=psd_whiten, low_frequency_cutoff=40, sigmasq = 1)
        peak = abs(snr).numpy().argmax()
        snrp = snr[peak]
        time = snr.sample_times[peak]

        signal_l1_scaled = signal_l1*snr_req[i][0] / abs(snrp) 


        final_scaled = add_noise_signal(noise, signal_l1_scaled)


        whiten = get_whiten (final_scaled)


        data = get_8s(whiten)

        my_dir = '/content/Positive Test DATA/' # write the file name in which you need to put the data
        name = 111000000+i+1
        np.save(my_dir + str(name), data)


In [ ]:

DATA_GENERATION(12288)

In [ ]:
import shutil
shutil.make_archive('/content/Positive_Test_DATA', 'zip', 'Positive Test DATA')

'/content/Positive_Test_DATA.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
!cp Positive_Test_DATA.zip '/content/gdrive/My Drive/'
!ls -lt '/content/gdrive/My Drive/' 

total 4805593
-rw------- 1 root root 1551385800 Dec  8 07:42  Positive_Test_DATA.zip
-rw------- 1 root root  387844632 Dec  8 07:20  Positive_Validation_DATA.zip
-rw------- 1 root root 1767534968 Dec  8 07:12  Positive_Training_DATA_2.zip
-rw------- 1 root root        151 Dec  7 17:51  FYP_GW_Project.gslides
-rw------- 1 root root  985257010 Dec  7 08:15  Positive_Training_DATA_1.zip
-rw------- 1 root root     157019 Dec  6 11:29 'Project Stage.docx'
-rw------- 1 root root  228727431 Dec  6 08:33  Positive_Training_DATA.zip
drwx------ 2 root root       4096 Nov 30 14:27 'Colab Notebooks'
-rw------- 1 root root        151 Sep 16 06:57  E-Governance.gslides
-rw------- 1 root root        151 Jul  9 15:33  Ajinkya_Kanse_192080011.gslides
drwx------ 2 root root       4096 Jul  6 14:12 'Links & Resources'
drwx------ 2 root root       4096 Jul  6 10:20  PAPERS
drwx------ 2 root root       4096 Jun 28 06:48  Books


In [ ]:
# N=2048*16  #N is number of samples, N=length/delta_t
# print(N)
# fs=2048.0 #fs is sampling frequnecy
# print(fs)
# length=16 #duration of segment
# print(length)
# delta_f=fs/N
# print(delta_f)
# f_samples = int(N/2+ 1)
# print(f_samples)
# f_lower=30
# print(f_lower)
# delta_t=1.0/2048
# print(delta_t)

32768
2048.0
16
0.0625
16385
30
0.00048828125
